In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

In [2]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [5]:
processor.encode("Hello, world!")

AttributeError: 'Qwen2_5_VLProcessor' object has no attribute 'encode'

In [3]:
import copy
import os
from qwenvl.train.argument import *
import torch


def set_processor(processor_args, processor):
  tokenizer = processor.tokenizer
  img_processor = processor.image_processor
  vid_processor = processor.video_processor
  
  tokenizer.padding_side = processor_args.padding_side
  tokenizer.model_max_length = processor_args.model_max_length
  img_processor.max_pixels = processor_args.image_max_pixels
  img_processor.min_pixels = processor_args.image_min_pixels
  img_processor.size["shortest_edge"] = processor_args.shortest_edge
  vid_processor.max_frame_pixels = processor_args.video_max_pixels
  vid_processor.min_frame_pixels = processor_args.video_min_pixels
  vid_processor.size['shortest_edge'] = processor_args.shortest_edge
  vid_processor.default_to_square = processor_args.video_default_to_square
  
  return copy.deepcopy(processor)


model_args, data_args, training_args, proc_args, vid_args = ModelArguments(), DataArguments(), TrainingArguments(), ProcessorArguments(), VideoProcessingArguments()
proc_args.model_max_length = 512

processor = transformers.AutoProcessor.from_pretrained(
    model_args.model_name_or_path,
    use_fast=True,
)
processor = set_processor(proc_args, processor)

In [4]:
import pathlib
from qwenvl.data.data_qwen import make_supervised_data_module
# if torch.distributed.get_rank() == 0:
#   model.visual.print_trainable_parameters()
#   model.model.print_trainable_parameters()
  
data_args.dataset_use = "openpmc_test"
data_module = make_supervised_data_module(processor=processor, data_args=data_args, vid_args=vid_args, num_proc=1)

vision tokens:   0%|          | 0/218126 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
from qwenvl.train.trainer import Trainer

training_args.deepspeed = "../scripts/zero3.json"

from transformers import Qwen2_5_VLForConditionalGeneration
os.makedirs(training_args.output_dir, exist_ok=True)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    attn_implementation='eager',
    torch_dtype=(torch.bfloat16 if training_args.bf16 else None),
)

model.config.use_cache = False

if training_args.gradient_checkpointing:
  if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()
  else:
    def make_inputs_require_grad(module, input, output):
      output.requires_grad_(True)
    model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)
trainer = Trainer(
    model=model, processing_class=processor, args=training_args, **data_module
)

if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
  print("checkpoint found, resume training")
  trainer.train(resume_from_checkpoint=True)
else:
  trainer.train()
  
trainer.save_state()
processor.save_pretrained(training_args.output_dir)

model.config.use_cache = True

safe_save_model_for_hf_trainer(
    trainer=trainer, output_dir=training_args.output_dir)

In [ ]:

from transformers import Qwen2_5_VLForConditionalGeneration
os.makedirs(training_args.output_dir, exist_ok=True)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    attn_implementation='eager',
    torch_dtype=(torch.bfloat16 if training_args.bf16 else None),
)

model.config.use_cache = False

if training_args.gradient_checkpointing:
  if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()
  else:
    def make_inputs_require_grad(module, input, output):
      output.requires_grad_(True)
    model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)
trainer = Trainer(
    model=model, processing_class=processor, args=training_args, **data_module
)

if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
  print("checkpoint found, resume training")
  trainer.train(resume_from_checkpoint=True)
else:
  trainer.train()
  
trainer.save_state()
processor.save_pretrained(training_args.output_dir)

model.config.use_cache = True

safe_save_model_for_hf_trainer(
    trainer=trainer, output_dir=training_args.output_dir)

In [3]:
import datasets

datasets.load_from_disk('/projects/cft_vlm/datasets/openpmc/data/dataset/test').select(range(1000)).save_to_disk('/projects/cft_vlm/datasets/openpmc/data/dataset/tiny')

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]